In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc,Pipeline
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.linalg import hankel

In [2]:
df_1 = pd.read_csv('~/data/ctown/dataset03.csv')
df_2 = pd.read_csv('~/data/ctown/dataset04.csv')
train_normal=pd.concat((df_1,df_2[df_2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack=df_2[df_2['ATT_FLAG']==1]

In [3]:
hankel = Hankel()
lag = 60
stride = 0.5

In [4]:
sensors=[col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']] 

In [5]:
scaler = StandardScaler()
X_normal=pd.DataFrame(data=scaler.fit_transform(train_normal.loc[:,sensors]),index=train_normal.index,columns=sensors)
X_normal

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,-1.810652,-0.840399,-1.489684,-1.438753,-0.125827,-0.299891,-2.102859,-0.192322,0.0,0.701055,...,-0.485068,0.969072,-0.467118,0.198329,-1.258253,0.754636,-1.256119,-0.150254,-1.034333,-0.359477
1,-1.891658,-0.867565,-0.824829,-1.366330,0.513682,0.702713,-1.754244,-0.182644,0.0,0.703361,...,-0.486352,1.186900,-0.470150,0.423304,-1.270108,0.771354,-1.267867,-0.140255,-1.042174,-0.371926
2,-1.968762,-0.883198,-0.091921,-0.577770,1.123162,0.731687,-1.280055,-0.249670,0.0,0.687387,...,-0.484604,1.376971,-0.470137,0.513959,-1.100738,0.955519,-1.097261,0.847781,-1.042107,-0.371820
3,-1.958116,-0.867475,0.693507,0.327115,1.636865,0.731687,-0.127985,-0.333661,0.0,0.667368,...,-0.451086,1.558821,-0.437744,0.705006,0.355822,-0.980130,0.350843,1.572762,-1.019584,-0.336060
4,-1.832527,-0.813155,1.524846,1.560727,1.028078,0.731687,1.359430,-0.355917,0.0,0.662064,...,-0.395961,1.780926,-0.383674,1.024098,0.396924,-0.972501,0.391854,1.653455,-0.990238,-0.289469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,-0.026020,-0.622019,-0.519318,-0.979847,1.456856,-2.422136,-1.347590,1.901208,0.0,-1.641849,...,-0.123946,-1.108734,-0.143233,0.189478,-1.360548,0.723200,-1.357927,-0.354694,-0.918065,-0.174880
12442,-0.367893,-0.492759,-1.152625,-1.202467,1.280082,-1.971590,-1.612310,1.805874,0.0,-1.641849,...,-0.018536,-1.277789,-0.040691,0.099141,0.213483,-1.362938,0.206009,-0.194622,-0.844885,-0.058695
12443,-0.643058,-0.356696,-1.844844,-2.000190,-0.066119,-1.295771,-1.732637,1.963771,0.0,-1.641849,...,-0.360210,0.735496,-0.340993,0.014118,0.000592,-1.492194,-0.005821,-0.347161,-0.965277,-0.249839
12444,-0.968254,-0.519972,-1.285178,-2.853569,-1.439516,-1.521044,-2.310208,1.895249,0.0,-1.641849,...,-0.730962,0.796970,-0.729189,2.778709,0.070801,-1.547303,0.064038,-0.283132,-1.130522,-0.512194


In [6]:
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [7]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')
df_test

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,04/01/17 00,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,...,90.54,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71,0.0
1,04/01/17 01,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,...,90.43,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73,0.0
2,04/01/17 02,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,...,91.72,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90,0.0
3,04/01/17 03,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,...,76.12,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90,0.0
4,04/01/17 04,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,...,75.68,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,31/03/17 20,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,...,70.07,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0
2085,31/03/17 21,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,...,68.29,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0
2086,31/03/17 22,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,...,87.08,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0
2087,31/03/17 23,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,...,87.85,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0


In [8]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
Y_test = test_combined.loc[:,'ATT_FLAG']

In [9]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [10]:
y_actual

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [11]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [12]:
sensor_predicitions = np.asarray(sensor_predicted)


In [13]:
sensor_predicitions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [15]:
predicted_label

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [16]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.788235294117647
Precision  0.78
Recall  0.8478260869565217
F1-score  0.8125


In [17]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5025991792065665 Median : 0.5058823529411764 Min : 0.43529411764705883 Max : 0.6235294117647059
precision - Mean: 0.6825258397932816 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.08897876643073813 Median : 0.08695652173913043 Min : 0.0 Max : 0.32608695652173914
f1 -        Mean: 0.1526845474696014 Median : 0.16 Min : 0.0 Max : 0.48387096774193544


# Multiple clusters: no threshold tuning

In [18]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=False,kscore_init='inertia')
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [19]:
sensor_predicitions = np.asarray(sensor_predicted)


In [20]:
sensor_predicitions

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [21]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [22]:
predicted_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [23]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.5764705882352941
Precision  0.5609756097560976
Recall  1.0
F1-score  0.71875


In [24]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5258549931600548 Median : 0.5176470588235295 Min : 0.4470588235294118 Max : 0.6588235294117647
precision - Mean: 0.5563626030972203 Median : 0.68 Min : 0.0 Max : 1.0
recall -    Mean: 0.18705763397371084 Median : 0.21739130434782608 Min : 0.0 Max : 0.43478260869565216
f1 -        Mean: 0.268658803783005 Median : 0.32786885245901637 Min : 0.0 Max : 0.5671641791044776


# Multiple clusters: threshold tuning (training attack is not in test data)

In [25]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [26]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [27]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=True,kscore_init='inertia')
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [28]:
sensor_predicitions = np.asarray(sensor_predicted)


In [29]:
sensor_predicitions

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [31]:
predicted_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [32]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969


In [33]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5714774281805746 Median : 0.5735294117647058 Min : 0.4411764705882353 Max : 0.7352941176470589
precision - Mean: 0.37644032164426 Median : 0.4 Min : 0.0 Max : 1.0
recall -    Mean: 0.13231756214915796 Median : 0.06896551724137931 Min : 0.0 Max : 0.3793103448275862
f1 -        Mean: 0.18467675979141693 Median : 0.12903225806451613 Min : 0.0 Max : 0.5499999999999999
